# 🚀 Forex Trading System - PPO Training (SIMPLIFIED VERSION)

**Goal**: Train AI model with realistic 3% risk strategy for profitable forex trading

**Strategy**:
- ✅ **SIMPLIFIED**: Single-pair trading (USD/JPY only)
- ✅ 3% risk per trade with 1:1.5 reward ratio
- ✅ $10,000 account, dynamic position sizing
- ✅ PPO reinforcement learning with 200k step checkpoints
- ✅ Immediate reward feedback to encourage trading

**Expected Training Time**: 12-24 hours total (200k steps per session)

**Key Improvement**: Simplified to single pair to make learning easier and see trades happening!

## 📦 Setup and Dependencies

In [ ]:
# Install required packages
!pip install stable-baselines3==2.1.0
!pip install sb3-contrib==2.1.0  # For RecurrentPPO with LSTM
!pip install gymnasium==0.29.1
!pip install ta-lib==0.4.28
!pip install torch torchvision torchaudio
!pip install tensorboard
!pip install loguru
!pip install PyYAML
!pip install pandas numpy matplotlib scikit-learn
!pip install quantstats

import warnings
warnings.filterwarnings('ignore')

print("✅ All dependencies installed!")
print("🧠 LSTM-based RecurrentPPO ready for temporal learning")

In [ ]:
# Check GPU and system info
import torch
import os
import psutil

print("🎮 System Information:")
print(f"   Current directory: {os.getcwd()}")
print(f"   Available RAM: {psutil.virtual_memory().available / 1e9:.1f} GB")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU Available: {gpu_name}")
    print(f"   GPU Memory: {gpu_memory:.1f} GB")
else:
    print("❌ No GPU available - training will be extremely slow!")
    print("   Go to Runtime → Change Runtime Type → Hardware Accelerator → GPU")

<cell_type>markdown</cell_type>## 📁 Create Project Structure and Upload Files

**Required Files to Upload:**
1. `src/data_manager.py` - Data processing and indicators
2. `src/environment_single.py` - **SIMPLIFIED** Single-pair trading environment
3. `training/train_ppo.py` - PPO training script
4. `config/training.yaml` - Training configuration
5. `requirements_colab.txt` - Additional requirements
6. **Optional**: Your CSV file: `USDJPY_M15.csv` (only need one pair now!)

In [ ]:
# Create required directories
import os

dirs = [
    'src', 'training', 'config', 'data', 
    'logs', 'logs/tensorboard', 'models', 
    'models/checkpoints', 'models/final'
]

for dir_name in dirs:
    os.makedirs(dir_name, exist_ok=True)
    print(f"📁 Created directory: {dir_name}")

print("\n📋 Upload your project files to these locations:")
print("   • data_manager.py → /content/src/")
print("   • environment_single.py → /content/src/")  # CHANGED
print("   • train_ppo.py → /content/training/")
print("   • training.yaml → /content/config/")
print("   • requirements_colab.txt → /content/" )
print("   • USDJPY_M15.csv (optional) → /content/data/")  # CHANGED

print("\n⚠️ IMPORTANT: Make sure all files are uploaded before proceeding!")

In [ ]:
# Install additional requirements if file exists
if os.path.exists('/content/requirements_colab.txt'):
    print("📦 Installing additional requirements...")
    !pip install -r requirements_colab.txt
    print("✅ Additional requirements installed")
else:
    print("ℹ️ No requirements_colab.txt found - using base packages only")

In [ ]:
# Verify file uploads
required_files = {
    'src/data_manager.py': 'Data processing module',
    'src/environment_single.py': 'Single-pair trading environment',  # CHANGED
    'training/train_ppo.py': 'PPO training script',
    'config/training.yaml': 'Training configuration'
}

print("🔍 Checking required files:")
all_files_present = True

for file_path, description in required_files.items():
    if os.path.exists(file_path):
        size_kb = os.path.getsize(file_path) / 1000
        print(f"✅ {file_path} - {description} ({size_kb:.1f} KB)")
    else:
        print(f"❌ {file_path} - {description} - FILE MISSING!")
        all_files_present = False

# Check optional CSV file (CHANGED - only USDJPY now)
csv_file = 'data/USDJPY_M15.csv'
print("\n🔍 Checking optional CSV file:")
if os.path.exists(csv_file):
    size_mb = os.path.getsize(csv_file) / 1e6
    print(f"✅ {csv_file} - USD/JPY data ({size_mb:.1f} MB)")
    csv_available = True
else:
    print(f"ℹ️ {csv_file} - Not uploaded (will use sample data)")
    csv_available = False

print(f"\n📊 Summary:")
print(f"   Required files: {'✅ All present' if all_files_present else '❌ Missing files'}")
print(f"   CSV file: {'✅ Uploaded' if csv_available else 'ℹ️ Using sample data'}")

if not all_files_present:
    print("\n⚠️ STOP: Upload missing files before continuing!")
else:
    print("\n🎯 Ready to proceed with training!")

## 🧪 Test Data Pipeline

In [ ]:
# Test data loading
import sys
import pandas as pd
import numpy as np

# Add paths
sys.path.append('/content/src')
sys.path.append('/content/training')

print("📊 Testing data pipeline...")

try:
    from data_manager import DataManager
    print("✅ DataManager imported successfully")
    
    # Initialize DataManager
    dm = DataManager()
    print("✅ DataManager initialized")
    
    # Check for CSV files
    csv_files = {
        'EURUSD': '/content/data/EURUSD_M15.csv',
        'AUDCHF': '/content/data/AUDCHF_M15.csv', 
        'USDJPY': '/content/data/USDJPY_M15.csv'
    }
    
    available_csvs = {}
    for pair, file_path in csv_files.items():
        if os.path.exists(file_path):
            available_csvs[pair] = file_path
    
    if available_csvs:
        print(f"\n📈 Loading {len(available_csvs)} CSV files...")
        try:
            # Load CSV data using DataManager
            data = dm.load_csv_data(available_csvs)
            print(f"✅ Real CSV data loaded for {len(data)} pairs!")
            use_real_data = True
        except Exception as e:
            print(f"⚠️ CSV loading failed: {e}")
            print("📊 Falling back to sample data...")
            data = dm.get_multi_pair_data()
            use_real_data = False
    else:
        print("📊 No CSV files found - using sample data...")
        data = dm.get_multi_pair_data()
        use_real_data = False
    
    # Display data summary
    print(f"\n📊 Data Summary ({'Real CSV' if use_real_data else 'Sample Data'}):")
    total_candles = 0
    for pair, df in data.items():
        features = len([col for col in df.columns if col not in ['Symbol']])
        candles = len(df)
        total_candles += candles
        date_range = f"{df.index[0]} to {df.index[-1]}" if hasattr(df.index, 'min') else "N/A"
        print(f"   {pair}: {candles:,} candles, {features} features ({date_range})")
    
    print(f"\n✅ Total: {total_candles:,} candles across {len(data)} pairs")
    
except Exception as e:
    print(f"❌ Data pipeline test failed: {e}")
    import traceback
    traceback.print_exc()
    print("\n🔧 Make sure all required files are uploaded correctly!")

## 🏗️ Test Trading Environment

In [ ]:
# Test trading environment
print("🏗️ Testing single-pair trading environment...")

try:
    from environment_single import SinglePairForexEnv  # CHANGED
    print("✅ Environment module imported")

    # Create environment with available data
    if 'data' in locals():
        # Get USDJPY data only
        usdjpy_data = data.get('USDJPY', list(data.values())[0])
        
        env = SinglePairForexEnv(usdjpy_data, initial_balance=10000.0)
        print("✅ Single-pair environment created with USD/JPY data")

        # Display environment info
        print(f"   Currency pair: USD/JPY")
        print(f"   Action space: {env.action_space}")
        print(f"   Actions: 3 (HOLD, BUY, SELL)")
        print(f"   Observation space: {env.observation_space.shape}")
        print(f"   Initial balance: ${env.initial_balance:,}")
        print(f"   Risk per trade: {env.risk_per_trade_pct * 100}%")

        # Quick environment test
        obs, info = env.reset()
        print(f"\n🧪 Environment test:")
        print(f"   Observation shape: {obs.shape}")
        print(f"   Initial balance: ${info['balance']:.2f}")

        # Test a few random actions
        for i in range(5):
            action = env.action_space.sample()  # Returns 0, 1, or 2
            obs, reward, done, truncated, info = env.step(action)

            action_name = ['HOLD', 'BUY', 'SELL'][action]
            print(f"   Step {i+1}: {action_name}, Reward: {reward:.1f}, Balance: ${info['balance']:.2f}, Has position: {info['has_position']}")

            if done:
                break

        print("\n✅ Environment test completed successfully!")
        print(f"   The agent is {('TRADING!' if env.episode_stats['total_trades'] > 0 else 'not trading yet')}")

    else:
        print("❌ No data available for environment test")

except Exception as e:
    print(f"❌ Environment test failed: {e}")
    import traceback
    traceback.print_exc()

## 🚀 Start PPO Training

**⚠️ IMPORTANT**: 
- Training runs in 200k step chunks (2-4 hours each)
- Total target: 500k steps (3 sessions max)
- Auto-saves checkpoints to handle Colab quotas
- Do NOT close browser during training!

In [ ]:
# Start or resume PPO training
print("🚀 STARTING PPO TRAINING WITH LSTM POLICY")
print("=" * 60)
print("🧠 Architecture: RecurrentPPO with LSTM (temporal learning)")
print("📊 Reward: Sharpe ratio-based (risk-adjusted)")
print("💰 Position sizing: True 3% risk ($300 per trade)")
print("⚡ Improvements: Transaction costs in reward, slippage, 30% max drawdown")
print("")
print("🎉 Training on:", 'Your CSV files' if use_real_data else 'Sample data')
print("")
print("=" * 60)

try:
    # Import training module
    from train_ppo import ForexTrainer, find_latest_checkpoint
    print("✅ Training module imported")
    
    # Check for existing checkpoints
    latest_checkpoint = find_latest_checkpoint()
    
    if latest_checkpoint:
        print(f"\n🔍 Found existing checkpoint: {os.path.basename(latest_checkpoint)}")
        print("📈 Resuming training from previous session...")
        resume_training = True
    else:
        print("\n🆕 Starting fresh training session...")
        resume_training = False
    
    # Initialize trainer
    trainer = ForexTrainer()
    print("✅ Trainer initialized")
    
    # Start training with appropriate data
    if use_real_data and available_csvs:
        print(f"📈 Training with {len(available_csvs)} CSV files...")
        if resume_training:
            trainer.train(csv_files=available_csvs, resume_from=latest_checkpoint)
        else:
            trainer.train(csv_files=available_csvs)
    else:
        print("📊 Training with sample data...")
        if resume_training:
            trainer.train(resume_from=latest_checkpoint)
        else:
            trainer.train()
    
    print("\n🎉 Training session completed successfully!")
    print("\n📥 IMPORTANT - Download these files:")
    print("   📁 /content/models/checkpoints/ - All checkpoint files")
    print("   📁 /content/models/final/ - Final model (if training complete)")
    print("   📁 /content/logs/ - Training logs and analytics")
    print("")
    print("🔄 For next session: Upload checkpoint files back to Colab to resume!")
    
except Exception as e:
    print(f"❌ Training failed: {e}")
    import traceback
    traceback.print_exc()
    
    # Check for emergency checkpoints
    emergency_files = []
    if os.path.exists('/content/models/checkpoints'):
        for file in os.listdir('/content/models/checkpoints'):
            if 'emergency' in file.lower():
                emergency_files.append(file)
    
    if emergency_files:
        print(f"\n💾 Emergency checkpoints available:")
        for file in emergency_files:
            print(f"   📄 {file}")
        print("📥 Download these files to resume training later!")

## 📈 Monitor Training Progress

In [ ]:
# Load TensorBoard for monitoring
%load_ext tensorboard
%tensorboard --logdir /content/logs/tensorboard --host 0.0.0.0 --port 6006

print("📈 TensorBoard loaded!")
print("\n📊 Key metrics to monitor:")
print("   • ep_rew_mean: Episode rewards (should trend upward)")
print("   • win_rate: Trade win rate (target >60%)")
print("   • avg_profit: Average profit per trade (target >$20)")
print("   • total_trades: Number of trades executed")
print("   • balance_progression: Account balance over time")
print("")
print("🎯 Good signs:")
print("   ✅ Rewards trending from negative to positive")
print("   ✅ Win rate stabilizing above 60%")
print("   ✅ Consistent trade execution (not all HOLD actions)")
print("   ✅ Balance growing steadily over episodes")

## 🔄 Resume Training (Day 2+)

**Instructions for resuming training:**
1. Upload checkpoint files to `/content/models/checkpoints/`
2. Run the cell below to continue training
3. Training will automatically detect and resume from latest checkpoint

In [ ]:
# Resume training from uploaded checkpoint
print("🔄 RESUME TRAINING SYSTEM")
print("=" * 40)

# Check for uploaded checkpoint files
checkpoint_dir = '/content/models/checkpoints'
if os.path.exists(checkpoint_dir):
    checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith('forex_ppo_chunk_')]
    
    if checkpoint_files:
        print(f"✅ Found {len(checkpoint_files)} checkpoint files:")
        for file in sorted(checkpoint_files):
            file_path = os.path.join(checkpoint_dir, file)
            size_mb = os.path.getsize(file_path) / 1e6
            print(f"   📄 {file} ({size_mb:.1f} MB)")
        
        # Find latest checkpoint
        latest_file = max(checkpoint_files, key=lambda x: os.path.getmtime(os.path.join(checkpoint_dir, x)))
        latest_path = os.path.join(checkpoint_dir, latest_file)
        print(f"\n🔍 Latest checkpoint: {latest_file}")
        
        # Resume training
        print("\n🚀 Resuming training...")
        try:
            from train_ppo import ForexTrainer
            
            trainer = ForexTrainer()
            
            # Resume with appropriate data
            if 'available_csvs' in locals() and available_csvs:
                trainer.train(csv_files=available_csvs, resume_from=latest_path)
            else:
                trainer.train(resume_from=latest_path)
            
            print("\n✅ Training session completed!")
            
        except Exception as e:
            print(f"❌ Resume training failed: {e}")
            import traceback
            traceback.print_exc()
            
    else:
        print("❌ No checkpoint files found in checkpoints directory!")
        print("📤 Please upload your checkpoint files first.")
        
else:
    print("❌ Checkpoints directory not found!")
    print("📁 Creating directory - please upload checkpoint files.")
    os.makedirs(checkpoint_dir, exist_ok=True)

In [ ]:
# TensorBoard Visualization (for detailed analysis)
%load_ext tensorboard
%tensorboard --logdir /content/logs/tensorboard --host 0.0.0.0 --port 6006

print("📈 TensorBoard loaded!")
print("\n🔍 KEY METRICS TO CHECK IN TENSORBOARD:")
print("\n1. SCALARS TAB - Look for these metrics:")
print("   📊 forex/win_rate - Should trend upward to >50%")
print("   💰 forex/profit_factor - Target: >1.5")
print("   📉 forex/max_drawdown - Should stay <30%")
print("   🎯 eval/mean_reward - Should become positive")
print("   📈 train/policy_loss - Should decrease")
print("   📉 train/value_loss - Should stabilize")
print("   🧠 train/explained_variance - Target: >0.8")
print("\n2. LOOK FOR TRENDS:")
print("   ✅ GOOD: Metrics improving steadily over time")
print("   ⚠️  PLATEAU: Metrics flat for 100k+ steps")
print("   ❌ BAD: Metrics degrading or very unstable")
print("\n3. DECISION GUIDE:")
print("   • Trending up → Continue training")
print("   • Plateau at good values → Start evaluation")
print("   • Plateau at poor values → Adjust hyperparameters")

In [ ]:
# Quick Metrics Check from Latest Metadata File
import yaml
import os
import glob

print("📊 FINAL TRAINING METRICS CHECK")
print("=" * 60)

# Find all metadata files
checkpoint_dir = '/content/models/checkpoints'
metadata_files = glob.glob(f'{checkpoint_dir}/*metadata*.yaml')

if metadata_files:
    # Get the most recent metadata file (excluding emergency)
    regular_metadata = [f for f in metadata_files if 'emergency' not in f]
    
    if regular_metadata:
        latest_metadata = max(regular_metadata, key=os.path.getmtime)
        
        with open(latest_metadata, 'r') as f:
            metadata = yaml.safe_load(f)
        
        print(f"✅ Loaded: {os.path.basename(latest_metadata)}")
        print(f"\n📈 PERFORMANCE METRICS:")
        print(f"   Win Rate: {metadata.get('win_rate', 'N/A')}%")
        print(f"   Profit Factor: {metadata.get('profit_factor', 'N/A')}")
        print(f"   Max Drawdown: {metadata.get('max_drawdown', 'N/A')}%")
        print(f"   Total Trades: {metadata.get('total_trades', 'N/A')}")
        print(f"   Goal Trades: {metadata.get('total_goal_trades', 'N/A')}")
        print(f"   Total Steps: {metadata.get('timesteps', 'N/A'):,}")
        
        print(f"\n🎯 TARGET COMPARISON:")
        win_rate = metadata.get('win_rate', 0)
        pf = metadata.get('profit_factor', 0)
        dd = metadata.get('max_drawdown', 100)
        
        win_status = "✅" if win_rate >= 50 else "⏳" if win_rate >= 45 else "❌"
        pf_status = "✅" if pf >= 1.5 else "⏳" if pf >= 1.2 else "❌"
        dd_status = "✅" if dd <= 30 else "⚠️"
        
        print(f"   {win_status} Win Rate: {win_rate}% (Target: >50%)")
        print(f"   {pf_status} Profit Factor: {pf} (Target: >1.5)")
        print(f"   {dd_status} Max Drawdown: {dd}% (Target: <30%)")
        
        print(f"\n📊 RECOMMENDATION:")
        if win_rate >= 50 and pf >= 1.5 and dd <= 30:
            print("   🎉 ALL TARGETS MET!")
            print("   → Start model evaluation and backtesting")
        elif win_rate >= 45 and pf >= 1.2:
            print("   ⏳ CLOSE TO TARGETS!")
            print("   → Consider 1-2 more training chunks (200k-400k steps)")
        else:
            print("   🔄 NEEDS MORE TRAINING")
            print("   → Continue for 2-4 more chunks or review hyperparameters")
    else:
        print("❌ Only emergency metadata found - upload regular checkpoint files")
else:
    print("❌ No metadata files found!")
    print("📤 Upload your checkpoint files to check metrics")

## 📊 Check Training Metrics

**Use this section to evaluate your completed training before deciding whether to:**
- ✅ Start evaluation (if targets met)
- ⏳ Continue training (if close to targets)
- 🔄 Adjust hyperparameters (if not improving)

## 📊 Training Results Analysis

In [ ]:
# Analyze training results
import glob
import json
from datetime import datetime

print("📊 TRAINING RESULTS ANALYSIS")
print("=" * 50)

# Check training progress
checkpoint_files = glob.glob('/content/models/checkpoints/forex_ppo_chunk_*')
if checkpoint_files:
    # Extract chunk numbers to determine progress
    chunk_numbers = []
    for file_path in checkpoint_files:
        try:
            filename = os.path.basename(file_path)
            # Extract chunk number from filename like 'forex_ppo_chunk_1_20231125_143022'
            parts = filename.split('_')
            if len(parts) >= 4:
                chunk_num = int(parts[3])
                chunk_numbers.append(chunk_num)
        except (ValueError, IndexError):
            continue
    
    if chunk_numbers:
        max_chunk = max(chunk_numbers)
        total_steps = max_chunk * 200000  # 200k steps per chunk
        progress_percent = min(total_steps / 500000 * 100, 100)  # Target 500k steps
        
        print(f"✅ Training Progress:")
        print(f"   Completed chunks: {max_chunk}/3")
        print(f"   Total steps: {total_steps:,}/500,000")
        print(f"   Progress: {progress_percent:.1f}%")
        print(f"   Checkpoint files: {len(checkpoint_files)}")
    else:
        print("⚠️ Could not parse checkpoint progress")
else:
    print("❌ No checkpoint files found")

# Check for final model
final_model_path = '/content/models/final/forex_ppo_model.zip'
if os.path.exists(final_model_path):
    size_mb = os.path.getsize(final_model_path) / 1e6
    print(f"\n🎉 TRAINING COMPLETE!")
    print(f"   Final model: {size_mb:.1f} MB")
    print(f"   Ready for download and deployment!")
elif chunk_numbers and max(chunk_numbers) >= 3:
    print(f"\n✅ Training target reached (3 chunks completed)")
    print(f"   Check /content/models/checkpoints/ for final model")
else:
    remaining = 3 - (max(chunk_numbers) if chunk_numbers else 0)
    print(f"\n⏳ Training in progress")
    print(f"   Estimated {remaining} more sessions needed")

# Check analytics files
analytics_files = glob.glob('/content/logs/*analytics*.csv')
if analytics_files:
    print(f"\n📈 Analytics files available: {len(analytics_files)}")
    for file_path in analytics_files[-3:]:  # Show last 3
        filename = os.path.basename(file_path)
        size_kb = os.path.getsize(file_path) / 1000
        print(f"   📄 {filename} ({size_kb:.1f} KB)")

# Check TensorBoard logs
tb_dir = '/content/logs/tensorboard'
if os.path.exists(tb_dir) and os.listdir(tb_dir):
    tb_files = len([f for f in os.listdir(tb_dir) if os.path.isfile(os.path.join(tb_dir, f))])
    print(f"\n📊 TensorBoard logs: {tb_files} files")
    print(f"   Available for performance analysis")

print(f"\n📅 Analysis completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 📦 Download Training Results

In [ ]:
# Create downloadable package with all training results
import zipfile
import glob
from datetime import datetime

def create_training_package():
    """Create comprehensive download package."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    package_name = f"forex_training_results_{timestamp}.zip"
    
    print(f"📦 Creating training package: {package_name}")
    
    with zipfile.ZipFile(package_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        files_added = 0
        
        # Add checkpoint files
        checkpoint_files = glob.glob('/content/models/checkpoints/forex_ppo_chunk_*')
        for file_path in checkpoint_files:
            arcname = f"checkpoints/{os.path.basename(file_path)}"
            zipf.write(file_path, arcname)
            files_added += 1
        print(f"   ✅ Added {len(checkpoint_files)} checkpoint files")
        
        # Add metadata files
        metadata_files = glob.glob('/content/models/checkpoints/*metadata*.yaml')
        for file_path in metadata_files:
            arcname = f"checkpoints/{os.path.basename(file_path)}"
            zipf.write(file_path, arcname)
            files_added += 1
        print(f"   ✅ Added {len(metadata_files)} metadata files")
        
        # Add final model if exists
        final_model = '/content/models/final/forex_ppo_model.zip'
        if os.path.exists(final_model):
            zipf.write(final_model, "final_model/forex_ppo_model.zip")
            files_added += 1
            print("   ✅ Added final model")
        
        # Add analytics files
        analytics_files = glob.glob('/content/logs/*analytics*.csv')
        for file_path in analytics_files:
            arcname = f"analytics/{os.path.basename(file_path)}"
            zipf.write(file_path, arcname)
            files_added += 1
        print(f"   ✅ Added {len(analytics_files)} analytics files")
        
        # Add configuration files
        config_files = ['/content/config/training.yaml']
        for file_path in config_files:
            if os.path.exists(file_path):
                arcname = f"config/{os.path.basename(file_path)}"
                zipf.write(file_path, arcname)
                files_added += 1
        print(f"   ✅ Added configuration files")
        
        # Add recent TensorBoard logs (sample)
        tb_files = glob.glob('/content/logs/tensorboard/**/*', recursive=True)
        tb_added = 0
        for file_path in tb_files[:20]:  # Limit to 20 most recent files
            if os.path.isfile(file_path):
                rel_path = os.path.relpath(file_path, '/content/logs')
                zipf.write(file_path, f"logs/{rel_path}")
                tb_added += 1
        print(f"   ✅ Added {tb_added} TensorBoard log files")
        
        files_added += tb_added
    
    return package_name, files_added

# Create and download package
try:
    package_file, total_files = create_training_package()
    package_size_mb = os.path.getsize(package_file) / 1e6
    
    print(f"\n📊 Package Summary:")
    print(f"   📦 File: {package_file}")
    print(f"   📏 Size: {package_size_mb:.1f} MB")
    print(f"   📄 Files: {total_files} total")
    
    # Auto-download
    try:
        from google.colab import files
        print(f"\n⬇️ Starting automatic download...")
        files.download(package_file)
        print(f"✅ Download initiated!")
    except Exception as download_error:
        print(f"⚠️ Auto-download failed: {download_error}")
        print(f"📥 Manual download: Right-click {package_file} in file browser")
    
    print(f"\n🎯 Next Steps:")
    print(f"   1. 📥 Download completes to your computer")
    print(f"   2. 📁 Extract the ZIP file")
    print(f"   3. 📋 Review analytics CSV files for performance metrics")
    print(f"   4. 🔄 Upload checkpoints to resume training (if not complete)")
    print(f"   5. 🚀 Deploy final model to your local inference system")
    
except Exception as e:
    print(f"❌ Package creation failed: {e}")
    import traceback
    traceback.print_exc()

## 🔧 Troubleshooting Guide

### Common Issues and Solutions:

**1. "Required files missing"**
- ✅ Upload all files to correct directories:
  - `src/data_manager.py` 
  - `src/environment.py`
  - `training/train_ppo.py`
  - `config/training.yaml`

**2. "No GPU available"**
- ✅ Go to Runtime → Change Runtime Type → Hardware Accelerator → GPU
- ✅ Restart runtime and re-run setup cells

**3. "Training fails to start"**
- ✅ Check all required files are uploaded correctly
- ✅ Verify Python modules import without errors
- ✅ Check error messages in training output

**4. "Colab disconnects during training"**
- ✅ Training auto-saves every 200k steps
- ✅ Download checkpoint files after each session
- ✅ Upload checkpoints and resume next day

**5. "CSV files not loading"**
- ✅ Upload CSV files to `/content/data/` folder
- ✅ Use exact filenames: `EURUSD_M15.csv`, `AUDCHF_M15.csv`, `USDJPY_M15.csv`
- ✅ Training will use sample data if CSV files unavailable

**6. "Poor training performance"**
- ✅ Check TensorBoard metrics (rewards should trend upward)
- ✅ Verify win rate >60% and consistent trade execution
- ✅ Review analytics CSV files for detailed trade analysis

### Support Files Format:

**Expected CSV format:**
```
Time,Open,High,Low,Close,Volume
2021-11-18 22:15:00,1.13714,1.1372,1.137,1.13716,438
2021-11-18 22:30:00,1.13716,1.1375,1.1371,1.13728,502
```

**requirements_colab.txt example:**
```
finta==1.3
yfinance==0.2.18
scikit-learn==1.3.0
```